In [1]:
import pandas as pd
import xgboost as xgb
import category_encoders as ce
import numpy as np
import glob
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import time
import datetime
import lightgbm as lgbm
from sklearn.model_selection import train_test_split
import time
from contextlib import contextmanager
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.metrics import mean_squared_log_error
from rgf.sklearn import RGFRegressor as RGF
from sklearn.ensemble import ExtraTreesClassifier

In [2]:
csv_file = '学歴フィルターが学生の企業や社員へのイメージに与える影響（回答） - フォームの回答 1.csv'

In [33]:
normal_dic = {
    'とてもそう思う' : 5,
    'そう思う' : 4,
    'どちらともいえない' : 3,
    'そう思わない' : 2,
    '全くそう思わない' : 1
}

reverse_dic = {
    'とてもそう思う' : 1,
    'そう思う' : 2,
    'どちらともいえない' : 3,
    'そう思わない' : 4,
    '全くそう思わない' : 5
}

#   '自身の待遇についてどう感じましたか。'
other_dic1 = {
    '非常に優遇されていると感じた' : 1,
    'やや優遇されていると感じた' : 2,
    'どちらともいえない' : 3,
    'やや冷遇されていると感じた' : 4,
}
#  '「学歴フィルター」を感じたことで、その企業へのイメージに変化はありましたか。'
other_dic2 = {
    '企業へのイメージが非常に良くなった' : 5,
    '企業へのイメージがやや良くなった' : 4,
    '企業へのイメージは変化しなかった' : 3,
    '企業へのイメージがやや悪くなった' : 2,
    '企業へのイメージが非常に悪くなった' : 1,
}

# 反転項目のリスト
reverse_cols = [
    '5段階のうち当てはまるものをお選びください。 [よい大学に入れた人はそれだけ努力したのだからほめられるべきである]',
    '5段階のうち当てはまるものをお選びください。 [有名大学の学生は熾烈な受験戦争を勝ち抜いた人なのだから社会から認められても当然だと思う]',
    '5段階のうち当てはまるものをお選びください。 [何をするにしても基礎学力は重要なので学歴を判断するのも一理ある]',
    '5段階のうち当てはまるものをお選びください。 [学歴は個人の努力と能力次第で決まるので学歴社会は公平だと思う]',
    '5段階のうち当てはまるものをお選びください。 [学歴社会は一生懸命勉強すれば誰しもに成功のチャンスが与えられる平等な社会といえる]',
    '5段階のうち当てはまるものをお選びください。 [家柄で将来が決まる社会よりは学歴社会の方がよい]'
]
# いいえの人は使わないやつ(とりあえず)
filter_col = '「学歴フィルター」の存在を感じた企業はありましたか。なお、「学歴フィルター」とは、企業が採用過程において主に大学名などの学歴を基準に選考し、一定水準に満たない大学の学生は足切りすることを指します。（なお、自身が足切りされたかや程度に関わらず「学歴フィルター」の存在を感じた場合は「はい」を選択してください。）'

# まだ不必要な行(定性情報)
drop_cols = [
    '性別を教えてください。',
    '2022年3月にいずれを卒業・修了予定ですか。',
    '在籍している大学名を教えてください。',
    '専攻を教えてください。',
    'タイムスタンプ',
    'スコア',
    'Unnamed: 50'
]

# 0~10評価の11段階評価
eval_11_cols = [
    'その企業への現在の好感度はどの程度ですか。好感度が低い状態を0、高い状態を10とした時にもっとも近い数値をお選びください。',
    'その企業で実際に働いている社員の印象はどの程度ですか。好感度が低い状態を0、高い状態を10とした時にもっとも近い数値をお選びください。'
]

# 企業、社員イメージのカラム、Yになりうる
corporate_image_cols = [
        '「学歴フィルター」を感じたことで、その企業へのイメージに変化はありましたか。',
        'その企業の現在のイメージについて5段階評価をしてください。 [消極的である]',
        'その企業の現在のイメージについて5段階評価をしてください。 [活気がない]',
        'その企業の現在のイメージについて5段階評価をしてください。 [親しみにくい]',
        'その企業の現在のイメージについて5段階評価をしてください。 [成長力がない]',
        'その企業の現在のイメージについて5段階評価をしてください。 [つめたい]',
        'その企業の現在のイメージについて5段階評価をしてください。 [不安定である]',
        'その企業の現在のイメージについて5段階評価をしてください。 [粗野である]',
        'その企業の現在のイメージについて5段階評価をしてください。 [不活発である]',
        'その企業の現在のイメージについて5段階評価をしてください。 [良心的でない]',
        'その企業の現在のイメージについて5段階評価をしてください。 [信頼できない]',
        'その企業の現在のイメージについて5段階評価をしてください。 [地味である]',
        'その企業の現在のイメージについて5段階評価をしてください。 [多様性がない]',
        'その企業の現在のイメージについて5段階評価をしてください。 [感情的である]',
        'その企業の現在のイメージについて5段階評価をしてください。 [無個性である]',
        'その企業の現在のイメージについて5段階評価をしてください。 [センスがない]',
        'その企業の現在のイメージについて5段階評価をしてください。 [保守的である]',
        'その企業の現在のイメージについて5段階評価をしてください。 [経営者が優れていない]',
        'その企業の現在のイメージについて5段階評価をしてください。 [優秀な人材が少ない]',
        'その企業への現在の好感度はどの程度ですか。好感度が低い状態を0、高い状態を10とした時にもっとも近い数値をお選びください。',
        'その企業で実際に働いている社員の印象はどの程度ですか。好感度が低い状態を0、高い状態を10とした時にもっとも近い数値をお選びください。'
]

# 選考のときの内容Xになりうる
selection_image_cols = [
#        '自身の待遇についてどう感じましたか。',
       '周りの学生の待遇と比較した際の自身の気持ちについて当てはまるものをお選びください。 [嫉妬心を感じた]',
       '周りの学生の待遇と比較した際の自身の気持ちについて当てはまるものをお選びください。 [ねたましさを感じた]',
       '周りの学生の待遇と比較した際の自身の気持ちについて当てはまるものをお選びください。 [怒りを感じた]',
       '周りの学生の待遇と比較した際の自身の気持ちについて当てはまるものをお選びください。 [うらやましくなった]',
       '周りの学生の待遇と比較した際の自身の気持ちについて当てはまるものをお選びください。 [不安になった]',
       '周りの学生の待遇と比較した際の自身の気持ちについて当てはまるものをお選びください。 [悲しくなった]',
       '周りの学生の待遇と比較した際の自身の気持ちについて当てはまるものをお選びください。 [自尊心を傷つけられた]'
]

# 学歴に対するイメージXになりうる
colledge_image_cols = [
       '5段階のうち当てはまるものをお選びください。 [よい大学に入れた人はそれだけ努力したのだからほめられるべきである]',
       '5段階のうち当てはまるものをお選びください。 [有名大学の学生は熾烈な受験戦争を勝ち抜いた人なのだから社会から認められても当然だと思う]',
       '5段階のうち当てはまるものをお選びください。 [何をするにしても基礎学力は重要なので学歴を判断するのも一理ある]',
       '5段階のうち当てはまるものをお選びください。 [学歴は個人の努力と能力次第で決まるので学歴社会は公平だと思う]',
       '5段階のうち当てはまるものをお選びください。 [学歴はやはりその人の能力をある程度表していると思う]',
       '5段階のうち当てはまるものをお選びください。 [学歴社会は一生懸命勉強すれば誰しもに成功のチャンスが与えられる平等な社会といえる]',
       '5段階のうち当てはまるものをお選びください。 [家柄で将来が決まる社会よりは学歴社会の方がよい]',
       '5段階のうち当てはまるものをお選びください。 [よい大学を出ているからといって実社会で通用するとは限らない]',
       '5段階のうち当てはまるものをお選びください。 [学歴で人柄まで判断されるのは不合理である]',
       '5段階のうち当てはまるものをお選びください。 [たとえ有名大学を出ていなくても社会で成功する道はいくらでもある]',
       '5段階のうち当てはまるものをお選びください。 [学歴で人間の価値がはかられることには問題がある]',
       '5段階のうち当てはまるものをお選びください。 [学歴によって将来が決まるのは納得がいかない]',
       '5段階のうち当てはまるものをお選びください。 [学歴の低い者が学歴の高い者を凌ぐことは現実的に困難である]',
       '5段階のうち当てはまるものをお選びください。 [日本の大学の序列には合理的根拠がほとんどない]'
]

normal_columns = corporate_image_cols + selection_image_cols + colledge_image_cols
reverse_columns = reverse_cols
other_col1 = '自身の待遇についてどう感じましたか。'
other_col2 = '「学歴フィルター」を感じたことで、その企業へのイメージに変化はありましたか。'
replace_rule = ((normal_columns, normal_dic), (reverse_columns, reverse_dic), (other_col1, other_dic1), (other_col2, other_dic2))

# どのカラムはどのルールで変換[[columns, dictionary], [], ...]
def clean_csv(csv_path, replace_rule):
    df = pd.read_csv(csv_path)
    for rule in replace_rule:
        df[rule[0]] = df[rule[0]].replace(rule[1])
    df.to_csv('cleaned.csv')
    return df

In [34]:
df = clean_csv(csv_file, replace_rule)

In [32]:
df

,タイムスタンプ,性別を教えてください。,2022年3月にいずれを卒業・修了予定ですか。,在籍している大学名を教えてください。,専攻を教えてください。,「学歴フィルター」の存在を感じた企業はありましたか。なお、「学歴フィルター」とは、企業が採用過程において主に大学名などの学歴を基準に選考し、一定水準に満たない大学の学生は足切りすることを指します。（なお、自身が足切りされたかや程度に関わらず「学歴フィルター」の存在を感じた場合は「はい」を選択してください。）,自身の待遇についてどう感じましたか。,周りの学生の待遇と比較した際の自身の気持ちについて当てはまるものをお選びください。 [嫉妬心を感じた],周りの学生の待遇と比較した際の自身の気持ちについて当てはまるものをお選びください。 [ねたましさを感じた],周りの学生の待遇と比較した際の自身の気持ちについて当てはまるものをお選びください。 [怒りを感じた],...,5段階のうち当てはまるものをお選びください。 [家柄で将来が決まる社会よりは学歴社会の方がよい],5段階のうち当てはまるものをお選びください。 [よい大学を出ているからといって実社会で通用するとは限らない],5段階のうち当てはまるものをお選びください。 [学歴で人柄まで判断されるのは不合理である],5段階のうち当てはまるものをお選びください。 [たとえ有名大学を出ていなくても社会で成功する道はいくらでもある],5段階のうち当てはまるものをお選びください。 [学歴で人間の価値がはかられることには問題がある],5段階のうち当てはまるものをお選びください。 [学歴によって将来が決まるのは納得がいかない],5段階のうち当てはまるものをお選びください。 [学歴の低い者が学歴の高い者を凌ぐことは現実的に困難である],5段階のうち当てはまるものをお選びください。 [日本の大学の序列には合理的根拠がほとんどない],Unnamed: 49,Unnamed: 50
0,2021/11/10 12:03:04,男性,学部,あ,文・人文学系,はい,1,5.0,5.0,5.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN
1,2021/11/10 22:24:04,男性,学部,北海道大学,経済・経営・商学系,はい,1,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN
2,2021/11/11 10:55:19,男性,学部,北海道大学,経済・経営・商学系,はい,1,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN
3,2021/11/11 13:01:13,男性,学部,北海道大学,経済・経営・商学系,いいえ,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021/11/11 13:08:51,男性,学部,ぽくだい,法・政治学系,はい,2,3.0,3.0,3.0,...,4.0,4.0,3.0,3.0,4.0,4.0,3.0,4.0,NaN,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,2021/11/29 1:58:12,女性,学部,横浜市立大学,経済・経営・商学系,はい,1,1.0,1.0,1.0,...,4.0,5.0,5.0,4.0,3.0,2.0,2.0,3.0,NaN,NaN
123,2021/11/29 6:35:52,女性,学部,筑波大学,文・人文学系,いいえ,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124,2021/11/29 7:26:29,女性,大学院（修士・博士）,広島大学,文・人文学系,いいえ,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,2021/11/29 9:03:05,男性,学部,横浜国立大学,経済・経営・商学系,いいえ,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
